In [15]:
import numpy as np
import pandas as pd
from datetime import datetime
import json
import requests

import psycopg2
import psycopg2.extras

In [16]:
def building_data_sequences(data_X, data_Y, timesteps):
    #generate data sequence with timesteps
    X, y_MPNxP = [], [] 
    for i in range(len(data_X) - timesteps + 1):
        sequence = data_X[i:(i + timesteps), :]
        target = data_Y[i + timesteps - 1]
        X.append(sequence)
        y_MPNxP.append(target)
    return np.array(X), np.array(y_MPNxP)

In [17]:
def data_extractor(asset,cols=['DCP','DNCP','OPCP','HPCP','LPCP','CPCP','ACPCP','VTCP','MPN5P']):
    # The credentials to conect to the database
    hostname = 'database-1.ctzm0hf7fhri.eu-central-1.rds.amazonaws.com'
    database = 'dyDATA_new'
    username = 'postgres'
    pwd = 'Proc2023awsrdspostgresql'
    port_id = 5432
    conn = None
    asset_script="SELECT * FROM "+'\"'+"ASSET_"+asset+'\"'+".features_targets_input_view WHERE features_targets_input_view."+'\"'+"cleaned_raw_features_environment_PK"+'\"'+ "= 4"
    # Here we select the active financial asset from the financial asset list table
    try:
        with psycopg2.connect(
            host = hostname,
            dbname = database,
            user = username,
            password = pwd,
            port = port_id
        ) as conn:
            dataframe = pd.read_sql(asset_script,conn)
    except Exception as error:
        conn.close()
        return error
    finally:
        if conn is not None:
            conn.close()
    dataframe = dataframe.filter(regex='|'.join(cols),axis=1)
    
    for i,j in zip(cols,dataframe.columns):
        dataframe.rename(columns={j:i},inplace=True)
    print(dataframe.tail())

    return dataframe

In [18]:
df = data_extractor(asset='MSFT')
df['DCP'] = pd.to_datetime(df['DCP'])
df.set_index('DCP', inplace=True) 

             DCP     DNCP    OPCP    HPCP    LPCP    CPCP   ACPCP        VTCP  \
6084  2024-03-11  45362.0  403.76  405.68  401.26  404.52  404.52  16093600.0   
6085  2024-03-12  45363.0  407.62  415.57  406.79  415.28  415.28  22427700.0   
6086  2024-03-13  45364.0  418.10  418.18  411.45  415.10  415.10  17090500.0   
6087  2024-03-14  45365.0  420.24  427.82  417.99  425.22  425.22  34113100.0   
6088  2024-03-15  45366.0  419.29  422.60  412.79  416.42  416.42  45049800.0   

       MPN5P  
6084  415.57  
6085  415.57  
6086  415.57  
6087  415.57  
6088  415.57  


/tmp/ipykernel_54487/1301242928.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(asset_script,conn)


In [19]:
df.tail()

,DNCP,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,MPN5P
DCP,,,,,,,,
2024-03-11,45362.0,403.76,405.68,401.26,404.52,404.52,16093600.0,415.57
2024-03-12,45363.0,407.62,415.57,406.79,415.28,415.28,22427700.0,415.57
2024-03-13,45364.0,418.10,418.18,411.45,415.10,415.10,17090500.0,415.57
2024-03-14,45365.0,420.24,427.82,417.99,425.22,425.22,34113100.0,415.57
2024-03-15,45366.0,419.29,422.60,412.79,416.42,416.42,45049800.0,415.57


In [20]:
features = ['OPCP', 'HPCP', 'LPCP', 'CPCP', 'ACPCP', 'VTCP'] 
target = 'MPN5P'

df = df[features + [target]]

train_start = '2022-01-01'
train_end = '2023-12-30'
test_start = '2024-01-01'
test_end = datetime.today()

train = df.loc[(df.index >= train_start) & (df.index <= train_end)]
test = df.loc[(df.index >= test_start) & (df.index <= test_end)]

In [21]:
X_train = train.drop("MPN5P", axis=1)
y_train = train["MPN5P"]
X_test = test.drop("MPN5P", axis=1)
y_test = test["MPN5P"]

train_index = X_train.index
test_index = X_test.index

In [22]:
X_train_array =  X_train.to_numpy('float64')
X_test_array = X_test.to_numpy('float64')
y_train_array = y_train.to_numpy('float64')
y_test_array = y_test.to_numpy('float64')

In [23]:
X_train_seq, y_train_seq = building_data_sequences(X_train_array,y_train_array,10)
X_test_seq, y_test_seq = building_data_sequences(X_test_array,y_test_array,10)

In [24]:
X_train_seq_array = X_train_seq.tolist()
y_train_seq_array = y_train_seq.tolist()

In [25]:
len(y_train_seq_array)

492

In [26]:
predict = json.dumps(True)
timesteps = 3
model_case_version_main_target_code = 5
iteration = 2
apply_hyperparameter_tuning = json.dumps(False)
learning_rate=0.0005
batch_size=64
n_epochs=2
attenuated_padding_value=1

bot_config = {
    'X_train': X_train_seq.tolist(),
    'y_train': y_train_seq.tolist(),
    'X_test': X_test_seq.tolist(),
    'y_test': y_test_seq.tolist(),
    'predict': predict,
    'timesteps': timesteps,
    'apply_hyperparameter_tuning': apply_hyperparameter_tuning,
    'model_case_version_main_target_code': model_case_version_main_target_code,
    'iteration': iteration,
    'model_name': 'MPN5P_LTM_3_timesteps',
    'learning_rate':learning_rate,
    'batch_size': batch_size,
    'n_epochs': n_epochs,
    'attenuated_padding_value': attenuated_padding_value,
    'endpoint': 'http://0.0.0.0:4000/graphql'
}

In [27]:
def fit_LTM_Bot(bot_config):
    query = f"""
        query {{
            fitLongShortTermMemory(
                X_train: {bot_config['X_train']}
                y_train: {bot_config['y_train']}
                X_test: {bot_config['X_test']}
                y_test: {bot_config['y_test']}
                predict: {bot_config['predict']}
                timesteps: {bot_config['timesteps']}
                apply_hyperparameter_tuning: {bot_config['apply_hyperparameter_tuning']}
                model_case_version_main_target_code: {bot_config['model_case_version_main_target_code']}
                iteration: {bot_config['iteration']}
                model_name: {json.dumps(bot_config['model_name'])}
                learning_rate: {bot_config['learning_rate']}
                batch_size: {bot_config['batch_size']}
                n_epochs: {bot_config['n_epochs']}
                attenuated_padding_value: {bot_config['attenuated_padding_value']}
            ){{
                success,
                error,
                train_predictions,
                test_predictions,
                model_path
            }}
        }}
    """

    endpoint = bot_config['endpoint']
    headers = {
            'Accept-Encoding': 'gzip, deflate, br',
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Connection': 'keep-alive',
            'DNT': '1'
        }

    response = requests.post(endpoint,json={"query":query},headers=headers).json()
    return response

In [28]:
fit_response = fit_LTM_Bot(bot_config)
print(fit_response)

{'data': {'fitLongShortTermMemory': {'error': None, 'model_path': '"/home/ubuntu/LTM-Long-short_Term_Memory.Bot-API_0.00/models/lstm_model.h5"', 'success': True}}}


In [29]:
fit_response['data']

{'fitLongShortTermMemory': {'error': None,
  'model_path': '"/home/ubuntu/LTM-Long-short_Term_Memory.Bot-API_0.00/models/lstm_model.h5"',
  'success': True}}